## 1. What are the main motivations for reducing a dataset's dimensionality? What are the main drawbacks?

Make it easier for the next models to work on lower dimension datasets, increasing speed, and avoiding overfitting resulting from high dimension datasets.

The main drawbacks are loosing a bit of information, result of reducing variance for example; this also leads to loosing information when recovering the initial dataset in the same proportion the variance was reduced. 

## 2. What is the curse of dimensionality?

It is the name given to the problems of having several dimensions in our dataset. These problems are:

1. Exponential growth in data volumen. As number of dimensions increase, we need much more data to represent the same information.
2. Data sparsity. As dimensions increase, the data density dicreases, making it harder for models to find patterns.
3. Computational cost. As dimensions increase, we need different algorithms to tackle these new dimensions.

## 3. Once a dataset's dimensionality has been reduced, is it possible to reverse the operation? If so, how? If not, why?

Yes, the operation can be reversed. For example, we execute a dot product, and so, we can reverse it by making the dot product of the transposed matrix. We will loose the same variance the selected hyperplane lets us reduce in the first operation